# Assignment: Credit Card Fraud Detection revised

we return to the credit card use case (last time we used Random Forests).


## Pre-Stage
Use your pre-processing from the last assignment as input.


In [1]:
#download data
#get the data
!git clone https://github.com/keuperj/DATA.git

Cloning into 'DATA'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 126 (delta 11), reused 39 (delta 11), pack-reused 87
Receiving objects: 100% (126/126), 185.56 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (86/86), done.


In [2]:
!unzip DATA/creditcard.csv.zip


Archive:  DATA/creditcard.csv.zip
  inflating: creditcard.csv          


In [25]:
import pandas as pd
data = pd.read_csv("creditcard.csv")
len(data)

284807

In [46]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix


In [27]:
# your preprocessing code
data.fillna(0, inplace=True) #replace NaN with zeros
data.replace([np.inf, -np.inf], 0, inplace=True) #also replace of inf values

data2 = data.copy()
data2 = data2[len(data)//2:]

x_labels_train = data2["Class"]

data2.drop(columns = ["Class"])
X_train=data2.to_numpy(dtype='float') #data type float

data3 = data.copy()
data3 = data3[:len(data)//2]

x_labels_test = data3["Class"]

data3.drop(columns = ["Class"])
X_test=data3.to_numpy(dtype='float') #data type float

#normalize data
from sklearn.preprocessing import normalize
X_train=normalize(X_train, axis=0)#normalize each variable
X_test=normalize(X_test, axis=0)#normalize each variable

# Task: Train a MLP-Network
* Train and evaluate the model
* Compare the results to the Random Forrest
* Tune the hyper-parameters (like number of layers, neurons per layer, learning-rate and number of epochs ) to optimize the results

See: https://keras.io/api/

In [28]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
model.fit(X_train, x_labels_train, epochs=10)

Epoch 1/10
4451/4451 [==============================] - 14s 3ms/step - loss: 0.0455 - accuracy: 0.9986
Epoch 2/10
4451/4451 [==============================] - 13s 3ms/step - loss: 7.2421e-04 - accuracy: 0.9997
Epoch 3/10
4451/4451 [==============================] - 14s 3ms/step - loss: 1.0443e-04 - accuracy: 1.0000
Epoch 4/10
4451/4451 [==============================] - 14s 3ms/step - loss: 1.2748e-05 - accuracy: 1.0000
Epoch 5/10
4451/4451 [==============================] - 15s 3ms/step - loss: 1.8298e-06 - accuracy: 1.0000
Epoch 6/10
4451/4451 [==============================] - 14s 3ms/step - loss: 2.5644e-07 - accuracy: 1.0000
Epoch 7/10
4451/4451 [==============================] - 13s 3ms/step - loss: 6.9207e-08 - accuracy: 1.0000
Epoch 8/10
4451/4451 [==============================] - 13s 3ms/step - loss: 1.2925e-08 - accuracy: 1.0000
Epoch 9/10
4451/4451 [==============================] - 13s 3ms/step - loss: 9.1018e-09 - accuracy: 1.0000
Epoch 10/10
4451/4451 [==================

In [31]:
test_loss, test_acc = model.evaluate(X_test,  x_labels_test, verbose=2)

print('\nTest accuracy:', test_acc)

4451/4451 - 9s - loss: 9.5387e-04 - accuracy: 0.9998 - 9s/epoch - 2ms/step

Test accuracy: 0.9998103976249695


In [41]:
predictions = model.predict(X_test)

4451/4451 [==============================] - 8s 2ms/step


In [42]:
i = 10
np.argmax(predictions[i])

0

In [43]:
x_labels_test[i]

0

In [47]:
confusion_matrix(x_labels_test, predictions)

ValueError: ignored